In [13]:
import os
from random import seed
from random import randrange #returns random numbers from a given range
from math import sqrt
from csv import reader #To read csv file without using read_csv of pandas

In [14]:
def load_csv(filename, header):
    # Initialize the dataset as list
    dset = list()
    
    # Open the dataset as readable file
    with open(filename, 'r') as file:
        
        #intialize the csv reader
        csv_reader = reader(file)
        
        #Append each row
        for row in csv_reader:
            
            if not row or not header:
                header = 1
                continue
            
            dset.append(row)
    
    return dset

#Convert a string column(if present) to float
def str_col_to_float(dset, col):
    
    for row in dset:
        row[col] = float(row[col].strip())

In [15]:
# Create dummy values for a string column. Convert the string to integer
def str_col_to_int(dset, col):
    
    #Store all the values of the column in a list
    class_values = [row[col] for row in dset]
    
    #Store the unique values
    uniq = set(class_values)
    
    #Create a dictionary with the lookup
    lookup = dict()
    for i, val in enumerate(uniq):
        lookup[val] = i
    
    #Now using the lookup change the string values to integer
    for row in dset:
        row[col] = lookup[row[col]]
    return lookup  

# k-fold cross validation
def cross_validation_split(dset, k):
    
    fold_size = int(len(dset)/k) #Observations in each fold
    
    dset_cpy = list(dset) #Make a copy of the dataset
    
    dset_split = list() #Split dataset
    
    for i in range(k):
        
        fold = list()
        
        while len(fold) < fold_size:
            
            fold.append(dset_cpy.pop(randrange(len(dset_cpy))))
        
        dset_split.append(fold)
        
    return dset_split


In [16]:
# Split the dataset into two based on a value
def test_split(col, val, dset):
    
    left, right = list(), list()
    
    for row in dset:
        
        if row[col] < val:
            
            left.append(row)
            
        else:
            
            right.append(row)
            
    return left, right


In [17]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    
    correct = 0.0
    
    for i in range(len(actual)):
        
        if actual[i] == predicted[i]:
            correct += 1
    
    return correct/float(len(actual)) * 100.0

#Evaluate function
def eval_algorithm(dset, algorithm, k, *args):
    
    folds = cross_validation_split(dset, k)
    scores = list()
    
    for fold in folds:
        
        #Create a copy of the data
        train_set = list(folds)
        
        #Remove the subsample
        train_set.remove(fold)
        
        #Create only one list
        train_set = sum(train_set, [])
        
        #Create test list
        test_set = list()
        for row in fold:
            
            row_cpy = list(row)
            test_set.append(row_cpy)
            row_cpy[-1] = None #Since the last column contains the dependent variable
            
        #Predicted values
        pred = algorithm(train_set, test_set, *args)

        #Actual Values
        actual = [row[-1] for row in fold]
        
        #Accuracy
        accuracy = accuracy_metric(actual, pred)
        
        #Append to score list
        scores.append(accuracy)
        
    return scores


In [18]:
# Calculate Gini Index
def gini_index(groups, class_values):
    
    size0 = float(len(groups[0]))
    size1 = float(len(groups[1]))
    gini = 0
    
    for group in groups:
        
        if len(group) == 0:
            continue
            
        gini_group = 0
        
        for class_val in class_values:
            
            proportion = [row[-1] for row in group].count(class_val) / float(len(group))
            gini_group += (proportion*(1.0 - proportion))
            
#         gini += gini_group * len(group) / (size0 + size1)
        gini += gini_group 
    
    return gini

In [19]:
# Get split. To get the best feature - split threshold combination. This is a greedy algorithm
def get_split(dset, n_features):
    
    class_values = list(set([row[-1] for row in dset]))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    
    # Sample of all features for random forest
    feat_sub = list()
    while len(feat_sub) < n_features:
        
        index = randrange(len(dset[0]) - 1)
        
        if index in feat_sub:
#             feat_sub.append(index)
            continue
        
        feat_sub.append(index)
    
    for feature in feat_sub:
        
        for row in dset:
            
            groups = test_split(feature, row[feature], dset)
            gini = gini_index(groups, class_values)
            
            if gini < b_score:
                b_index, b_value, b_score, b_groups = feature, row[feature], gini, groups
    
    # Return a dictionary
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

In [20]:
# Create terminal node value
def to_terminal(group):
    
    outcomes = [row[-1] for row in group]
    
    #returns the most common output value in a list of rows.
    return max(set(outcomes), key = outcomes.count)

# Create the child nodes recursively
def split(node, max_depth, min_size, n_features, depth):
    
    # Store node in left and right
    left, right = node['groups']
    del(node['groups'])
    
    # If either of the group contains no value then we should stop and treat it as terminal node
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    
    # If max depth is reached
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # Left Node Split
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
        
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
        
    # Right Node Split
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
        
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)
    
#Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    
    # Create the root node
    root = get_split(train, n_features)
    
    # Create child nodes
    split(root, max_depth, min_size, n_features, 1)
    
    return root


In [21]:
# Prediction of a data point. The terminal nodes contain the modal class and other nodes are all dictionaries. So we 
# need to find a way to navigate through the nodes according the given data point value and the values in each node. 
# This could be done recursively as as we created the split
def predict(node, row):
    
    if row[node['index']] < node['value']:
        
        if isinstance(node['left'], dict):
            
            return predict(node['left'], row)
            
        else:
            
            return node['left']
        
    else:
        
        if isinstance(node['right'], dict):
            
            return predict(node['right'], row)
                
        else:
            
            return node['right']
    

In [22]:
# Create a random subsample from the dataset with replacement
def subsample(dset, ratio):
    
    n_samp = round(len(dset)*ratio)
    
    sample = list()
    
    while len(sample) < n_samp:
        
        index = randrange(len(dset))
        
        sample.append(dset[index])
    
    return sample

In [23]:
from sklearn import datasets

In [24]:
dset = load_csv('/content/drive/My Drive/HR_Data.csv.txt', header = 0)

# convert string attributes to integers
for i in range(0, len(dset[0])-1):
    str_col_to_float(dset, i)

#Convert class column to integer
str_col_to_int(dset, len(dset[0])-1)

# Bagging all the trees and produce predictions
def bagging_pred(trees, row):
    
    predictions = [predict(tree, row) for tree in trees]
    
    return max(set(predictions), key = predictions.count)

In [25]:
# Random Forest Algorithm: Create multiple trees, subsample data and get scores
def random_forest(train_data, test_data, max_depth, min_size, sample_size, n_trees, n_features):
    
    trees = list()
    for i in range(n_trees):
            
            sample = subsample(train_data, sample_size)
            tree = build_tree(sample, max_depth, min_size, n_features)
            trees.append(tree)
    
    # Bag the results of all trees
    predictions = [bagging_pred(trees, row) for row in test_data]
    
    return(predictions)

seed(1)
#Evaluation Algorithm
n_folds = 3 #k fold validation
max_depth = 10 #Maximum allowable depth of tree
min_size = 1 #Minimum size of in node
sample_size = 1.0 #Full or sample
n_features = int(sqrt(len(dset[0])-1)) #Number of features considered 
n_trees = [1, 5] #Number of Trees for Random Forest

for n_tree in n_trees:
    scores = eval_algorithm(dset, random_forest, n_folds, max_depth, min_size, sample_size, n_tree, n_features)
    print('Trees: %d' % n_tree)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))


Trees: 1
Scores: [83.84458077709611, 84.04907975460122, 83.640081799591]
Mean Accuracy: 83.845%
Trees: 5
Scores: [83.640081799591, 85.2760736196319, 82.82208588957054]
Mean Accuracy: 83.913%
